In [1]:
import json

import geopandas
import pandas

places = pandas.read_csv("https://raw.githubusercontent.com/davidbailey/Notes/trunk/Places.csv")
places["Notes"] = places["Notes"].map(json.loads)
places["geometry"] = geopandas.tools.geocode(places["Name"] + ", " + places["Address"], timeout=10)["geometry"]

In [2]:
gdf = geopandas.GeoDataFrame(places, geometry="geometry", crs="EPSG:4326")
gdf = gdf[gdf["Notes"].map(lambda x: True if x.get("Favorite") else False)]
gdf = gdf[gdf["Notes"].map(lambda x: False if x.get("Closed") else True)]
gdf

,Name,Address,URL,Notes,geometry
7,Colori Kitchen,"2019 Venice Boulevard, Los Angeles, CA, 90006,...",https://www.colorikitchen.com,{'Favorite': 'True'},POINT (-118.41576 34.01364)
9,The Standing Room,"144 North Catalina Avenue, Redondo Beach, CA, ...",https://www.thestandingroomrestaurant.com,{'Favorite': 'True'},POINT (-118.38855 33.84239)
11,Lucky Bamboo Kitchen,"14323 Hawthorne Boulevard, Lawndale, CA 90260,...",NaN,{'Favorite': 'True'},POINT (-118.35203 33.88819)
12,Benny Tudino's,"622 Washington Street, Hoboken, NJ, 07030, Uni...",https://www.bennytudinos.com,{'Favorite': 'True'},POINT (-74.02904 40.74422)
24,King Cab Halal Food,"West 28th Street, New York, NY, United States",NaN,"{'Favorite': 'True', 'Address': 'False'}",POINT (-73.98035 40.60392)
...,...,...,...,...,...
116,3Fe,"32 Grand Canal Street Lower, Dublin 2, Ireland",http://www.3fe.com,{'Favorite': 'True'},POINT (-6.25011 53.33741)
119,Club Rodeo,"610 Coleman Avenue, San Jose, CA, United States",NaN,{'Favorite': 'True'},POINT (-121.90874 37.34145)
120,El Tarasco,"316 Rosecrans Avenue, Manhattan Beach, CA, 902...",NaN,{'Favorite': 'True'},POINT (-118.41727 33.90102)
121,Tatsu Ramen,"2123 Sawtelle Boulevard,Los Angeles, CA, 90025...",https://tatsuramen.com/tatsu-ramen-sawtelle/,{'Favorite': 'True'},POINT (-118.43687 34.04162)


In [3]:
gdf = gdf[gdf["geometry"].is_empty.map(lambda x: not x)]

import folium

folium_map = folium.Map()
tooltip = folium.features.GeoJsonTooltip(fields=["Name"])
geojson = folium.GeoJson(
    gdf[["Name", "geometry"]].to_json(),
    tooltip=tooltip
)
geojson.add_to(folium_map)

In [4]:
folium_map